In [159]:
import pandas as pd
import tqdm
import datetime
from datetime import timedelta
import os

# Učitavanje originalnih podataka 

In [160]:
dataset_path = '../data/raw/export_df.parquet'

In [161]:
data = pd.read_parquet(dataset_path)
data.head()


,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id
0,0,2015-07-01,2015-01-21,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313223,1077152
1,0,2015-07-01,2015-06-10,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313224,1017906
2,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896
3,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245
4,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703


#### Uklanjanje zapisa (redaka) gdje je zemlja gosta 0 

In [162]:
data = data[~(data['zemlja_gosta']=='0')]

#### Dodavanje stupca  ukupno_gostiju ; broj djece za svaki redak (djeca + odrasli)

In [163]:
data['ukupno_gostiju'] = data['broj_odraslih_gostiju'] + data['broj_djece_gostiju']
data['ukupno_gostiju'] = data['ukupno_gostiju'].astype('int')

#### Uklanjanje rezervacija u kojima nema gostiju, odnosno ukupan je broj gostiju 0

NOTE: Zašto ove retke uklanjamo sada, a ne nakon primjene statističke ili metode strojnog učenja? Jer ovdje govorimo o retcima u kojima je broj gostiju nula, što znači kako su ti podaci nevaljani: označen je dolazak gosta, odnosno očekujemo da je gost došao, ali je zbog nekog razloga krivo postavljen broj gostiju te možemo reći da su ovdje krivo uneseni podaci

In [164]:
data = data[data['ukupno_gostiju'] != 0]

#### Dodavanje stupca duljina boravka

In [165]:
data['duljina_boravka'] = data['datum_odjave'] - data['datum_dolaska']
data['duljina_boravka'] = data['duljina_boravka'].dt.days

#### Uklanjanje redaka gdje je duljina boravka 0

In [166]:
data = data[data['duljina_boravka']>0]

#### Dodavanje stupca raspon_dolazak_rezervacija

In [167]:
data['raspon_dolazak_rezervacija'] = (data['datum_dolaska'] - data['datum_kreiranja_rezervacije']).dt.days

#### Uklanjanje (u zaseban df) radaka s otkazanim rezervacijama

In [168]:
data_canceled = data[data['datum_otkazivanja_rezervacije'].notna()]
data_canceled = data_canceled.reset_index(drop=True)
data = data[~data['datum_otkazivanja_rezervacije'].notna()]
data = data.reset_index(drop=True)
data.head(5)

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896,1,1,49
1,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245,1,1,397
2,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703,2,2,360
3,0,2015-07-01,2015-04-21,2015-07-03,None,2,0.0,GBR,2,1,143.304560,Check-Out,1313228,1044263,2,2,71
4,0,2015-07-01,2015-06-25,2015-07-03,None,2,0.0,PRT,0,0,161.502814,Check-Out,1313229,1022260,2,2,6


In [169]:
data_canceled.head()

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-06-30,2015-07-04,2015-05-06,2,0.0,PRT,2,1,61.437192,Canceled,1313231,1075633,2,3,1
1,0,2015-07-01,2015-06-28,2015-07-04,2015-04-22,2,0.0,PRT,2,2,150.365262,Canceled,1313232,1056139,2,3,3
2,0,2015-07-01,2015-04-17,2015-07-05,2015-06-23,2,0.0,PRT,2,3,142.635814,Canceled,1313233,1020092,2,4,75
3,0,2015-07-01,2015-01-04,2015-07-08,2015-05-11,2,0.0,PRT,2,3,113.277032,Canceled,1313250,1018277,2,7,178
4,0,2015-07-01,2015-01-01,2015-07-11,2015-05-29,2,0.0,PRT,0,3,105.416363,Canceled,1313255,1035627,2,10,181


####  Uklanjanje stupaca: 
   - datum otkazivanja rezervacije (ovdje imamo samo neotkazane rezervacije)
   - datum odjave : jer je autokoreliran s stupcima datum dolaska i duljina boravka
   - broj_odraslih_gostiju	te broj_djece_gostiju uklanjamo kako smo dodali stupac ukupan broj gostiju
   - status rezervacije (sve su vrijednosti iste iste - 'Check-Out')
   - rezervacija_id - od 74553 zapisa imamo i 74553 unikatne vrijednosti identifikatora
    (NE ODBACUJEMO gost_id jer od 74553 zapisaimamo 69764 gosta, što znači da imamo ponavljajućih gostiju)
   - datum_kreiranja_rezervacije - dodan je stupac raspon_dolazak_rezervacija koji označava razliku dana dolaska i kreiranja rezervacije 
   

In [170]:
data = data.drop(columns = ['datum_otkazivanja_rezervacije','datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data_canceled = data_canceled.drop(columns = ['datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data.head()

,hotel_id,datum_dolaska,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,GBR,0,1,64.991345,1039896,1,1,49
1,0,2015-07-01,GBR,1,1,74.368897,1008245,1,1,397
2,0,2015-07-01,GBR,2,1,130.973278,1093703,2,2,360
3,0,2015-07-01,GBR,2,1,143.304560,1044263,2,2,71
4,0,2015-07-01,PRT,0,0,161.502814,1022260,2,2,6


#### Odvajanje na hotel 0 (Resort) te na hotel 1 (City)

In [171]:
dataResort = data[data['hotel_id'] == 0]
dataCity = data[data['hotel_id'] == 1]

assert len(dataResort) + len(dataCity) == len(data)

# Stvaranje podatkovnih okvira na dnevnoj, tjednoj, mjesečnoj i godišnjoj razini za oba hotela

## Hotel 1 (City Hotel)

### Dnevna razina

In [ ]:
daily_guest_counts_city_hotel = {}
for index, row in dataCity.iterrows():
    
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    
    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)

        if current_day not in daily_guest_counts_city_hotel: #init
            daily_guest_counts_city_hotel[current_day] = 0

        daily_guest_counts_city_hotel[current_day] += number_of_guests

NumberOfGuestsDailyCity = pd.DataFrame(list(daily_guest_counts_city_hotel.items()), columns=['Date', 'NumberOfGuests'])

In [ ]:
NumberOfGuestsDailyCity['Date'] = pd.to_datetime(NumberOfGuestsDailyCity['Date'])
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.sort_values(by='Date')
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.set_index('Date')
NumberOfGuestsDailyCity.head(5)

In [ ]:
NumberOfArrivalsDailyCity = dataCity.groupby('datum_dolaska').size()
NumberOfArrivalsDailyCity = NumberOfArrivalsDailyCity.resample('D').sum().to_frame(name='NumberOfArrivals')
NumberOfArrivalsDailyCity.head(5)

In [ ]:
NumberOfNewGuestsDailyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfNewGuestsDailyCity = NumberOfNewGuestsDailyCity.resample('d').sum().to_frame()
NumberOfNewGuestsDailyCity.head(5)

### Tjedna razina

In [ ]:
NumberOfGuestsWeeklyCity = NumberOfGuestsDailyCity.resample('W').sum()

In [ ]:
NumberOfArrivalsWeeklyCity =NumberOfArrivalsDailyCity.resample('W').sum()

In [ ]:
NumberOfNewGuestsWeeklyCity = NumberOfNewGuestsDailyCity.resample('W').sum()

### Mjesečna razina

In [ ]:
NumberOfGuestsMonthlyCity = NumberOfGuestsDailyCity.resample('ME').sum()

In [ ]:
NumberOfArrivalsMonthlyCity =NumberOfArrivalsDailyCity.resample('ME').sum()

In [ ]:
NumberOfNewGuestsMonthlyCity = NumberOfNewGuestsDailyCity.resample('ME').sum()

### Godišnja razina

In [ ]:
NumberOfGuestsYearlyCity = NumberOfGuestsDailyCity.resample('YE').sum()

In [ ]:
NumberOfArrivalsYearlyCity =NumberOfArrivalsDailyCity.resample('YE').sum()

In [ ]:
NumberOfNewGuestsYearlyCity = NumberOfNewGuestsDailyCity.resample('YE').sum()

## Hotel 0 (Resort Hotel)

### Dnevna razina

In [ ]:
daily_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    
    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)

        if current_day not in daily_guest_counts_resort_hotel: #init
            daily_guest_counts_resort_hotel[current_day] = 0

        daily_guest_counts_resort_hotel[current_day] += number_of_guests

NumberOfGuestsDailyResort = pd.DataFrame(list(daily_guest_counts_resort_hotel.items()), columns=['Date', 'NumberOfGuests'])

In [ ]:
NumberOfGuestsDailyResort['Date'] = pd.to_datetime(NumberOfGuestsDailyResort['Date'])
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.sort_values(by='Date')
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.set_index('Date')
NumberOfGuestsDailyResort.head(5)

In [ ]:
NumberOfArrivalsDailyResort = dataResort.groupby('datum_dolaska').size()
NumberOfArrivalsDailyResort = NumberOfArrivalsDailyResort.resample('D').sum().to_frame(name='NumberOfArrivals')
NumberOfArrivalsDailyResort.head(5)

In [ ]:
NumberOfNewGuestsDailyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfNewGuestsDailyResort = NumberOfNewGuestsDailyResort.resample('d').sum().to_frame()
NumberOfNewGuestsDailyResort.head(5)

### Tjedna razina

In [ ]:
NumberOfGuestsWeeklyResort = NumberOfGuestsDailyResort.resample('W').sum()

In [ ]:
NumberOfArrivalsWeeklyResort = NumberOfArrivalsDailyResort.resample('W').sum()

In [ ]:
NumberOfNewGuestsWeeklyResort = NumberOfNewGuestsDailyResort.resample('W').sum()

### Mjesečna razina

In [ ]:
NumberOfGuestsMonthlyResort = NumberOfNewGuestsDailyResort.resample('ME').sum()

In [ ]:
NumberOfArrivalsMonthlyResort = NumberOfArrivalsDailyResort.resample('ME').sum()

In [ ]:
NumberOfNewGuestsMonthlyResort = NumberOfNewGuestsDailyResort.resample('ME').sum()

### Godišnja razina

In [ ]:
NumberOfGuestsYearlyResort = NumberOfGuestsDailyResort.resample('YE').sum()

In [ ]:
NumberOfArrivalsYearlyResort = NumberOfArrivalsDailyResort.resample('YE').sum()

In [ ]:
NumberOfNewGuestsYearlyResort = NumberOfNewGuestsDailyResort.resample('YE').sum()

### Spremanje podatakovnih okvira (interim za sada)

In [176]:
DATA_PATH = '../data/interim'
DATA_HOTEL0_BASE_PATH = '../data/interim/hotel_0'
DATA_HOTEL1_BASE_PATH = '../data/interim/hotel_1'

In [177]:
data.to_parquet(os.path.join(DATA_PATH,"data_no_cancellations.parquet"))
data_canceled.to_parquet(os.path.join(DATA_PATH,"data_cancellations.parquet"))
dataCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"hotel0cleaned.parquet"))
dataResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"hotel1cleaned.parquet"))



In [ ]:
NumberOfGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_arrivals.parquet")) 
NumberOfNewGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_guests.parquet")) 
NumberOfArrivalsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_arrivals.parquet")) 
NumberOfNewGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_guests.parquet")) 
NumberOfArrivalsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_arrivals.parquet")) 
NumberOfNewGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_new_guests.parquet"))

In [ ]:
NumberOfGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_arrivals.parquet"))
NumberOfNewGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_guests.parquet"))
NumberOfArrivalsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_arrivals.parquet"))
NumberOfNewGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_guests.parquet"))
NumberOfArrivalsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_arrivals.parquet"))
NumberOfNewGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_new_guests.parquet"))